# Домашнее задание 2. Извлечение коллокаций + NER

## Дедлайн
23 ноября 2020 (понедельник) 23:59 мск

## Описание

Выберите корпус отзывов на товары одной из категорий Amazon:
http://snap.stanford.edu/data/web-Amazon-links.html

(В низу страницы по ссылке есть код для загрузки данных, можете им воспользоваться)

Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

**Важное замечание**: в задании приводятся примеры решений, вы можете их использовать!

1. (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. 
Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара.
Могут помочь заголовки и дополнительные данные с Amazon (Metadata [здесь](https://nijianmo.github.io/amazon/index.html))
Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

2. (2 балла) Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них): 
- написать правила с помощью [natasha/yargy](https://github.com/natasha/yargy)
- составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)

3. (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание.
Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

5. (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.
Должны получиться примерно такие группы:

```
watch 
---
stylish watch
good watches
great watch
love this watch
...
```

**Бонус** (2 балла): 
если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

## Формат выполнения

jupyter-ноутбук + укажите, какие данные брали — или положите их тоже в репозиторий.

[Корпус, который я брала](http://snap.stanford.edu/data/amazon/Cell_Phones_&_Accessories.txt.gz)

In [ ]:
!pip install simplejson

     |████████████████████████████████| 133kB 4.3MB/s 


In [ ]:
from tqdm.auto import tqdm

In [ ]:
import pandas as pd

In [ ]:
import gzip
import simplejson

def parse(filename):
    f = gzip.open(filename, 'rt')
    entry = {}
    for l in f:
        l = l.strip()
        colonPos = l.find(':')
        if colonPos == -1:
            yield entry
            entry = {}
          continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
    yield entry


all_data = []
for e in parse("Cell_Phones_&_Accessories.txt.gz"):
    all_data.append(simplejson.dumps(e))

In [ ]:
data = []
for el in all_data:
    data.append(eval(el))

In [ ]:
type(data[0])

dict

In [ ]:
data[0]

{'product/price': 'unknown',
 'product/productId': 'B000JVER7W',
 'product/title': 'Mobile Action MA730 Handset Manager - Bluetooth Data Suite',
 'review/helpfulness': '0/0',
 'review/profileName': 'A. Igoe',
 'review/score': '1.0',
 'review/summary': "Don't buy!",
 'review/text': "First of all, the company took my money and sent me an email telling me the product was shipped. A week and a half later I received another email telling me that they are sorry, but they don't actually have any of these items, and if I received an email telling me it has shipped, it was a mistake.When I finally got my money back, I went through another company to buy the product and it won't work with my phone, even though it depicts that it will. I have sent numerous emails to the company - I can't actually find a phone number on their website - and I still have not gotten any kind of response. What kind of customer service is that? No one will help me with this problem. My advice - don't waste your money!"

In [ ]:
data[0]['review/text']

"First of all, the company took my money and sent me an email telling me the product was shipped. A week and a half later I received another email telling me that they are sorry, but they don't actually have any of these items, and if I received an email telling me it has shipped, it was a mistake.When I finally got my money back, I went through another company to buy the product and it won't work with my phone, even though it depicts that it will. I have sent numerous emails to the company - I can't actually find a phone number on their website - and I still have not gotten any kind of response. What kind of customer service is that? No one will help me with this problem. My advice - don't waste your money!"

In [ ]:
texts = []
for t in tqdm(data):
    try:
        texts.append(t['review/text'])
    except KeyError:
        pass

In [ ]:
len(texts)

78930

### (3 балла) Предложите 3 способа найти упоминания товаров в отзывах.

#### Первый способ



---


Би-граммы и три-граммы для составления частотного словаря (по аналогии с семинаром)

In [ ]:
import gensim

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

Штука с семинара работала не так как нужно, поэтому я просто брала из [докумнетации](https://radimrehurek.com/gensim/models/phrases.html)

In [ ]:
for_bgram = [t.split() for t in texts]

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(for_bgram, min_count=5, threshold=50) # higher threshold fewer phrases.

In [ ]:
bigram_mod = []

for sent in tqdm(for_bgram):
    bg_sent = bigram[sent]
    bigram_mod.append(bg_sent)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


По аналогии с семинаром, но с основой документации

In [ ]:
def make_bigrams(texts):
    bigram_mod = []

    for sent in tqdm(texts):
        bg_sent = bigram[sent]
        bigram_mod.append(bg_sent)

    return bigram_mod


def make_trigrams(texts):
    tr_mod = []

    for doc in tqdm(texts):
        bg_sent = bigram[sent]
        tr_sent = trigram[bg_sent]
        tr_mod.append(tr_sent)
    
    return tr_mod 

In [ ]:
textb = make_bigrams(for_bgram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
trigram = gensim.models.Phrases(textb, threshold=50)  

In [ ]:
text_tr = make_trigrams(for_bgram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('english')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tf_vector(text, ran):
    # есть два варианта, как на это посмотреть: взять н-грамм ранж. от 1
    # (если мы хотим выделять в том числе штуки типа "телефон" и "телефон новый")

    # или просто инициализировать векторайзер без н-грамм ранж. так как у нас уже есть н-граммы
    
    # я посмотрю на оба варианта: сначала с н-грамм ранж., потом без него, строчку закомментриую
    
    # vectorizer = TfidfVectorizer(min_df=5, stop_words=stop, ngram_range=(1, ran))
    
    vectorizer = TfidfVectorizer(min_df=5, stop_words=stop)
    tfidf_matrix = vectorizer.fit_transform(" ".join(t) for t in text)
    text1_tfidf = vectorizer.transform([" ".join(text[0])])
    vocab = vectorizer.get_feature_names()
    tfidf = vectorizer.idf_
    top = list(sorted(zip(vocab, text1_tfidf.toarray()[0]), key=lambda x: x[1], reverse = True))
    for el in top[:40]:
        print(el)
    
    # найдем среди вхождений такие, где попадаются "_" при склейке
    for el in top[:40]:
        if '_' in str(el[0]):
            print("-->", el)

In [ ]:
# c n-gramm range
print('Биграммы')
tf_vector(textb, 2)

Биграммы
('email telling', 0.3897989724948018)
('telling', 0.25987968091140634)
('email', 0.18812526608003233)
('company', 0.1814718557261209)
('money', 0.15175306350503692)
('shipped', 0.14125414555252114)
('website still', 0.1379056145627919)
('company took', 0.13587804027839723)
('numerous emails', 0.13587804027839723)
('half later', 0.13412167554391607)
('items received', 0.1325724520536114)
('money sent', 0.1325724520536114)
('number website', 0.1325724520536114)
('actually find', 0.13118662441809426)
('company actually', 0.13118662441809426)
('customer_service one', 0.13118662441809426)
('another email', 0.12993299083160056)
('company buy', 0.12993299083160056)
('depicts', 0.12993299083160056)
('help problem', 0.12993299083160056)
('later received', 0.12993299083160056)
('one help', 0.12993299083160056)
('kind', 0.1283427894694591)
('went another', 0.1267609387503409)
('back went', 0.12345535052555505)
('still gotten', 0.12345535052555505)
('sent', 0.12082331327484438)
('received

In [ ]:
# без n-gramm range
print('Биграммы')
tf_vector(textb, 2)

Биграммы
('telling', 0.4209231694001017)
('email', 0.3047036342546513)
('company', 0.29392720662604205)
('money', 0.2457921305455607)
('shipped', 0.22878719270530262)
('depicts', 0.21045049046799563)
('kind', 0.207874865490354)
('sent', 0.19569568418242853)
('received', 0.17361377605576445)
('my_advice', 0.17133178480917874)
('actually', 0.16930604125431714)
('another', 0.15030877687215732)
('numerous', 0.1385803565420854)
('sorry', 0.13124216083693685)
('emails', 0.13119518389720833)
('mistake', 0.12661093019034073)
('response', 0.12653538389296784)
('product', 0.11727529028268827)
('gotten', 0.11721703892638537)
('items', 0.11687887256365365)
('waste', 0.11234179039325506)
('website', 0.11148761673247545)
('later', 0.10980672921858654)
('even_though', 0.10636850645929201)
('customer_service', 0.10367794069977619)
('help', 0.10288060730151811)
('half', 0.10092364655672013)
('finally', 0.0964894695274278)
('number', 0.09305478652614334)
('went', 0.09272834109892153)
--> ('my_advice', 0

In [ ]:
# c n-gramm range
print('Триграммы')
tf_vector(text_tr, 3)

Триграммы
('phone', 0.2785430072655778)
('earpiece', 0.18569533817705186)
('earpieces', 0.18569533817705186)
('earpieces one', 0.18569533817705186)
('one', 0.18569533817705186)
('anything', 0.09284766908852593)
('anything product', 0.09284766908852593)
('anything product would', 0.09284766908852593)
('bluetooth', 0.09284766908852593)
('bluetooth earpieces', 0.09284766908852593)
('bluetooth earpieces one', 0.09284766908852593)
('change', 0.09284766908852593)
('change anything', 0.09284766908852593)
('change anything product', 0.09284766908852593)
('charging', 0.09284766908852593)
('charging change', 0.09284766908852593)
('charging change anything', 0.09284766908852593)
('clarity', 0.09284766908852593)
('clarity outstanding', 0.09284766908852593)
('clarity outstanding even', 0.09284766908852593)
('color', 0.09284766908852593)
('color maybe', 0.09284766908852593)
('color maybe size', 0.09284766908852593)
('connects', 0.09284766908852593)
('connects phone', 0.09284766908852593)
('connects 

In [ ]:
# без n-gramm range
print('Триграммы')
tf_vector(text_tr, 3)

Триграммы
('phone', 0.4375949744936837)
('earpiece', 0.2917299829957891)
('earpieces', 0.2917299829957891)
('one', 0.2917299829957891)
('anything', 0.14586499149789456)
('bluetooth', 0.14586499149789456)
('change', 0.14586499149789456)
('charging', 0.14586499149789456)
('clarity', 0.14586499149789456)
('color', 0.14586499149789456)
('connects', 0.14586499149789456)
('disconnect', 0.14586499149789456)
('easily', 0.14586499149789456)
('even', 0.14586499149789456)
('idle', 0.14586499149789456)
('long_periods', 0.14586499149789456)
('maybe', 0.14586499149789456)
('noisy', 0.14586499149789456)
('number', 0.14586499149789456)
('outstanding', 0.14586499149789456)
('performance', 0.14586499149789456)
('places', 0.14586499149789456)
('plugged', 0.14586499149789456)
('product', 0.14586499149789456)
('size', 0.14586499149789456)
('time', 0.14586499149789456)
('tops', 0.14586499149789456)
('tried', 0.14586499149789456)
('unlike', 0.14586499149789456)
('would', 0.14586499149789456)
--> ('long_perio

**Метод: по словарю**

Такой метод в теории позволит выделить нам все нужнные сущности из текста (или тайтла) и составить словарь, на котором мы найдем все нужные н-граммы с сущностями дальше в текстах => искать колокации.

Для улучшения полуавтоматической собрки словаря мы могли бы **выделить** название товара с использованием description/feature из metadata для выделения нужных названий товаров (и проверять попадается ли часть этих данных в словаре --> своеобразная фильтрация).



**Достоинства**:


*   словарь можно апдейтить
*   сводим действия разметчика к минимуму, все делается получавтоматически



(На примере того, как попробовала выделить словарь: на триграммах мы можем получить информацию более точную, чем на би-граммах о специфике датасета (телефон, наушники, цвет, соединение и т.д.), но все еще не названия)


**Недостатком** (как и в семинаре) служит то, что качество 

*   (как и в семинаре) служит то, что качество не очень - очень много мусора. (На три-граммах этого меньше, но мусор есть и не всегда получилось бы выделить название товара.) Необходимо улучшать полуавтоматическое создание словаря, например, давать морфологическую информацию о тексте. Вообще, главное недостаток - очень много неоднозначности (чаще всего это просто частотные слова, но не сущности, которые нам нужны)
*   мы скорее хотим обучать все это на саммари или заголовках и искать в самих текстах. Но есть вероятность, что это будет работать плохо, так как в текстах может быть все написано не так, как в лейблах/саммари

#### Второй способ

**Метод: извлечения сущностей с помощью  rule-based подхода**


Построить необходимые правила и с помощью парсера, который по заготовкам выделял бы нужные элементы из текста, выделить нужные элементы.


**Иноформация, которая могла бы помочь:** по аналогии с предыдущим блоком могла бы помочь информация о фичах устройства (в меньшей степени) и его описании


**Достоинства**: 


*   Рзаметчик в состоянии ограничить область включаемых в правила параметров (в нашем случае - одна область, легко выделить какие-то конкретные параметры про устройства)
*   легкая формализация (все, что нам нужно - похожее друг на друга скорее всего)
*   в рамках одной тематики (устройств) все будет работает очень хорошо, так как основывается на четко выделенных критериях
*   будет возможность учитывать разную грамматическую информацию, если правило усложним





**Недостатки**: 



*   Разметчик может что-то упустить/включить что-то лишнее в правила, тогда картинка будет неполной, так как не удастся учесть все случаи

*   внутри отзывов может что-то не ловиться, так как там можно что-то упросить/не учитывать/сократить (когда отзывы пишут реальные люди)



#### Третий способ

**Метод: теггинг последовательности**

С помощью метода получится приписать каждому токену какой-то тег, с помощью чего мы могли бы выделить нужные токены из текста

**Иноформация, которая могла бы помочь:** тут мало отличий от предыдущих блоков --> нам могут помочь здесь саммари, фичи и описание.


**Достоинства**:



*   каждый тег будет учитывать предыдущие/следующие  (текуший объект и контекст)
*   для нас последовательность может быть очень важна в данной задаче, так как вряд ли в отзывах слова относящиеся к одному предмету будут сильно раскиданы про тексту (с другой стороны это может быть и минусом, но мне кажется, что это больше плюс в данном случае)




**Недостатки**: 


*   Так как изначально нет разметки по токенам, придется опираться на словарь, где теги бы хранились бы со словами, а всему остальному - приписать, например, части речи (или просто смотреть по частям речи, но это кажется слишком простым)
*   динамическое программирование --> последовательность тегов всегда будет разной



### (2 балла) Реализуйте один из предложенных вами способов.


---

Попробуем сделать следующее: использовать небольшой словарь (составим полуавтоматически: выберем самое хорошее, что может получить на биграммах/триграммах и тф-айдф и посмотрим вместе с этим на именованные сущности, которые выделяются автоматически) дескриптеров, которые потом расширим с помощью эмбеддингов (word2vec) и почистим его от того мусора, который нам не нужен, выбрав только то, что будет макисмально похоже на сущности данного корпуса.


Для этого пройдемся по тайтлам, чтобы вся информация о компаниях-производителях и их устройствах сохарнилась.


In [ ]:
# собирем датасет

title = []
helpfulness = []
score = []
summary = []

for t in tqdm(data):
    try:
        title.append(t['product/title'])
        helpfulness.append(t['review/helpfulness'])
        score.append(t['review/score'])
        summary.append(t['review/summary'])
    except KeyError:
        pass

In [ ]:
df = pd.DataFrame({'title': title, 'helpfulness': helpfulness, 
                   "score": score, "summary": summary, "text": texts})

In [ ]:
df.head()

,title,helpfulness,score,summary,text
0,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,1.0,Don't buy!,"First of all, the company took my money and se..."
1,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,5.0,Mobile Action Bluetooth Mobile Phone Tool Soft...,Great product- tried others and this is a ten ...
2,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,4.0,good,works real good....a little hard to set up...w...
3,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",4/5,4.0,No instructions included...,The price was right for this cable ($11.95+$4....
4,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",0/0,1.0,NOT A DATA CABLE,this is NOT a DATA CABLE this is only a USB ch...


In [ ]:
# уникальные тайтлы
un_titles = df.title.unique().tolist()

In [ ]:
un_titles[:3]

['Mobile Action MA730 Handset Manager - Bluetooth Data Suite',
 'USB Data Cable for Sony-Ericsson Z600, Z500, Z200, S700, P900, P802, P800, K700, T637, T630,T628, T618, T616, T610, T316, T310, T237, T226, T68i, T68, T62u, T39, R380 + Internet + Driver CD',
 'Motorola HT820 Stereo Bluetooth Headset with microphone , Wireless , 12 hours Music']

In [ ]:
len(un_titles)

7150

In [ ]:
# очистка текста

def clean_t(un_titles):
    for_title = []
    
    # '+' оставляем, так как может быть в марке телефона/устройства
    # числа не удаляем, так как может быть очень важным для каких-нибудь моделей
    marks = ["'", '!', '(', ')', '-', '[', ']', '{', '}', ';', 
            '?', '@', '#', '$', '%', ':', '"', ',', '.', '/', '\\',
            '^', '&', '*', '_']

    for t in un_titles:
        list_t = []
        new_t = t
        # удаляем пунктуацию
        for m in marks:
            new_t = new_t.replace(m, ' ').replace('  ', ' ')
        # удаляем стоп-слова
        new = new_t.split(' ')
        for word in new:
            if word not in stop:
                list_t.append(word)

        for_title.append(list_t)
        
    return for_title

In [ ]:
for_title = clean_t(un_titles)

In [ ]:
for_title[:1]

[['Mobile',
  'Action',
  'MA730',
  'Handset',
  'Manager',
  'Bluetooth',
  'Data',
  'Suite']]

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(for_title, min_count=5, threshold=50)

In [ ]:
textb_2 = make_bigrams(for_title)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
textb_2[4:8]

[['Belkin',
  'Executive',
  'Leather',
  'Case',
  'handheld',
  'carrying',
  'case',
  'F8D0402'],
 ['Samsung',
  'T809',
  'M620',
  'i607',
  'U420',
  'A303',
  'A717',
  'Car',
  'Charger',
  'CAD300MBEB'],
 ['For',
  'Motorola',
  'V9m',
  'V3_V3c',
  'V3i_V3m',
  'V3t_V3v',
  'Razr_Razor',
  'Leather',
  'Case',
  'Pouch',
  'Cover',
  'Magnetic_Button',
  'Expand',
  'Cover',
  'Belt',
  'Clip',
  'Black'],
 ['NEW',
  'UNLOCKED',
  'SAMSUNG',
  'X830',
  'PINK',
  'MP3',
  'CAMERA',
  'GSM',
  'CELL_PHONE']]

In [ ]:
tf_vector(textb_2, 3)

('handset_manager', 0.5157976312808434)
('suite', 0.5102109904576572)
('mobile_action', 0.45665984154559564)
('data', 0.44733991783367316)
('bluetooth', 0.25492417659649586)
('00', 0.0)
('001', 0.0)
('01', 0.0)
('02', 0.0)
('03', 0.0)
('04', 0.0)
('05', 0.0)
('06', 0.0)
('10', 0.0)
('100', 0.0)
('1000', 0.0)
('1000_mah', 0.0)
('1000mah', 0.0)
('100_genuine', 0.0)
('11', 0.0)
('1100', 0.0)
('1100mah', 0.0)
('110_240v', 0.0)
('12', 0.0)
('120', 0.0)
('1200', 0.0)
('120c', 0.0)
('1221', 0.0)
('1260', 0.0)
('1260_1261', 0.0)
('12_volt', 0.0)
('12v', 0.0)
('13', 0.0)
('14', 0.0)
('1400', 0.0)
('1400_mah', 0.0)
('1400mah', 0.0)
('15', 0.0)
('1500_mah', 0.0)
('1600', 0.0)
--> ('handset_manager', 0.5157976312808434)
--> ('mobile_action', 0.45665984154559564)
--> ('1000_mah', 0.0)
--> ('100_genuine', 0.0)
--> ('110_240v', 0.0)
--> ('1260_1261', 0.0)
--> ('12_volt', 0.0)
--> ('1400_mah', 0.0)
--> ('1500_mah', 0.0)


In [ ]:
trigram = gensim.models.Phrases(textb_2, threshold=50)

In [ ]:
text_tr_2 = make_trigrams(for_title)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
tf_vector(text_tr_2, 3)

('phone', 0.43301270189221935)
('earpiece', 0.2886751345948129)
('earpieces', 0.2886751345948129)
('one', 0.2886751345948129)
('anything', 0.14433756729740646)
('bluetooth', 0.14433756729740646)
('change', 0.14433756729740646)
('charging', 0.14433756729740646)
('clarity', 0.14433756729740646)
('color', 0.14433756729740646)
('connects', 0.14433756729740646)
('disconnect', 0.14433756729740646)
('easily', 0.14433756729740646)
('even', 0.14433756729740646)
('idle', 0.14433756729740646)
('long', 0.14433756729740646)
('maybe', 0.14433756729740646)
('noisy', 0.14433756729740646)
('number', 0.14433756729740646)
('outstanding', 0.14433756729740646)
('performance', 0.14433756729740646)
('periods', 0.14433756729740646)
('places', 0.14433756729740646)
('plugged', 0.14433756729740646)
('product', 0.14433756729740646)
('size', 0.14433756729740646)
('time', 0.14433756729740646)
('tops', 0.14433756729740646)
('tried', 0.14433756729740646)
('unlike', 0.14433756729740646)
('would', 0.14433756729740646)


In [533]:
# Запомним отсюда следующие слова (кажутся лучше всего относящимися к теме)
# и то, что можно выхватить глазами просмотрев инфо о товарах

from_b_gr = ['phone', 'earpiece', 'connect', 'bluetooth', 'item',
             'product', 'headset', 'battery', 'adapter',
             'flash', 'memory card', 'charger', 'watch', 'car vent',
             'camera']

Все получается очень неплохо: на н-граммах нам удалось выделить вот такие штуки, где видны нужные нам вещи. Теперь посмотрим на все это через призму именованных сущностей ддя выделения названий или других полезных штук.

Cмотреть будем на модель [spacy](https://explosion.ai/demos/displacy-ent), потому что она умеет видеть продукты (делала также как [тут](https://dev.by/news/nlp-is-fun))

In [ ]:
import spacy
import en_core_web_sm
model_sp = en_core_web_sm.load()

Берем именно такие лейблы (если проверить на указанном сайте, то можно увидеть, что иногда такие штуки выделяются как личность или произведение искусства)

In [ ]:
# нужные нам лейблы, которыми могут маркироваться ис

enti = []

lab = ['ORG', ' PRODUCT', 'WORK OF ART', 'PERSON', 'QUANTITY']

for doc in tqdm(un_titles):
    for ent in model_sp(doc).ents:
        if str(ent.label_) in lab:
            enti.append(str(ent.text.strip()))

In [ ]:
enti[:5]

['USB Data Cable',
 'Sony-Ericsson Z600',
 'T610',
 'Wireless',
 'Belkin Executive Leather Case']

In [ ]:
from collections import Counter

In [ ]:
cnt = Counter(enti)

In [ ]:
keys = list(dict(cnt).keys())

In [ ]:
# столько именованных сущностей выделилось
len(keys)

3985

Сначала я хотела брать топ-1000 ключей, но так как я выбираю только то, что напрямую может маркировать товар/организцию его выпускающую, то будет неуместно взять только часть, поэтому я рещила, что лучше взять все ключи (названия товаров) и дополнить этот словарь тем хорошим, что выдали нам н-граммы (тогда у нас будут и штуки типа "телефон" и штуки типа "iphone10")

In [534]:
# возьмем ключи и то, что мы выделили сами
entities = set(keys).union(set(from_b_gr)) 

Расширим с помощью ворд-ту век (возьмем модель по указанной ссылке)

In [ ]:
from gensim.models import phrases, word2vec

In [535]:
# сделаем на выделенных через н-граммы штуках

model = word2vec.Word2Vec(textb_2, size=50, min_count=5)

In [536]:
new_ent = []

for w in entities:
    if w in model:
        new_ent.append(model.wv.most_similar(w)[0][0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [537]:
# объединение сущностей
fin_ent_1 = list(set(entities).union(set(new_ent)))

Ограничим наш словарь по длине, чтобы не было очень длинных сущностей и, собственно, мусора, который перечислял бы кучу всего, что может привести к шумам

In [538]:
fin_ent = []

for el in fin_ent_1:
    if len(el) < 30:
        fin_ent.append(el)

Нам нужно сделать так, чтобы попадали именованные сущности в выборку, соответственно, кажется лишним приводить тексты к нижнему регистру, так как очень много названий (собственно упоминающих товар), а по условию задания нам необходимо это учитывать, поэтому оставляем так

In [515]:
clean_texts = clean_t(texts)

In [516]:
all_clean_texts = []

for t in clean_texts:
    new_t = ' '.join(t)
    all_clean_texts.append(new_t)

In [539]:
ent_in_texts = []
for text in tqdm(all_clean_texts):
    for_t = []
    for el in fin_ent:
        if el in text:
            for_t.append(el)
    ent_in_texts.append(for_t)

In [540]:
df['enteties'] = ent_in_texts

Очистим для следующего задания (я делала н-граммы без нижнего регистра, мне казалось очень выжным оставить код с верхним регистром, но на всякий случай пусть будет этот момент)

In [519]:
cl = []
for s in all_clean_texts:
    cl.append(s.lower())

In [541]:
cl_en = []
for i in ent_in_texts:
    n = []
    for el in i:
        n.append(el.lower())
    cl_en.append(n)

In [542]:
df['clean_text'] = cl
df['clean_enteties'] = cl_en

In [543]:
df.head()

,title,helpfulness,score,summary,text,enteties,clean_text,clean_enteties,bi_grams,tri_grams
0,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,1.0,Don't buy!,"First of all, the company took my money and se...","[A, W, item, phone, product]",first company took money sent email telling pr...,"[a, w, item, phone, product]","[(work, phone), (phone, even), (telling, produ...","[(product, work, phone), (phone, even, though)..."
1,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,5.0,Mobile Action Bluetooth Mobile Phone Tool Soft...,Great product- tried others and this is a ten ...,[product],great product tried others ten compared real e...,[product],"[(Great, product), (product, tried)]",[]
2,Mobile Action MA730 Handset Manager - Bluetoot...,0/0,4.0,good,works real good....a little hard to set up...w...,[],works real good little hard set web part doesn...,[],[],[]
3,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",4/5,4.0,No instructions included...,The price was right for this cable ($11.95+$4....,"[Sony Ericsson, A, W, CD, Ericsson, 5, phone, ...",the price right cable 11 95+ 4 94 s h compared...,"[sony ericsson, a, w, cd, ericsson, 5, phone, ...","[(Files, Sony Ericsson), (Sony Ericsson, Mobil...","[(Program, Files, Sony Ericsson), (Sony Ericss..."
4,"USB Data Cable for Sony-Ericsson Z600, Z500, Z...",0/0,1.0,NOT A DATA CABLE,this is NOT a DATA CABLE this is only a USB ch...,"[A, USB, charger]",not data cable usb charger dont buy,"[a, usb, charger]","[(CABLE, USB), (USB, charger), (USB, charger),...","[(DATA, CABLE, USB), (USB, charger, dont), (CA..."


Как видим, у нас получилось выделить упоминание товара в отзывах, но не везде с помощью такой полуавтоматической сборки сущности вообще нашлись. В задании указано, что нам нужно выделить н-граммы с названиями товара, причем название может быть странным: watch / galaxy watch. Поэтому самым лучшим решением мне кажется выделить все возможные сущности/навзвания и т.д. с учетом апдейта с инфо об устройствах и получить н-граммы с ними.

### (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [523]:
import string

In [544]:
# би-граммы для сущностей

def got_ner_grams(sentence, entities=ent_in_texts):
    
    all_grams = []
    new_data = zip(sentence, entities)
    
    for s, e in tqdm(new_data):
        text_bigrams = []
        new_s = s.replace('  ', ' ')
        sent = new_s.split(' ')
        for w in e:

            # чтобы были би-граммы допускаем отступы
            entit = str(' ' + w + ' ')

            if entit in new_s:
                start = new_s.find(entit)
                end = start + len(w) + 1
                # print(s.find(w), w, s)
                # print(s[start:end])
                right = new_s[:start].split(' ')
                # print(right)
                left = new_s[end+1:].split(' ')
                # print(left)
                bg_r = (right[-1], w)
                bg_l = (w, left[0])
                # print(bg_r, bg_l)

                # исключаем штуки, когда в не + сосед пустые
                if bg_l[1] != '' and bg_r[0] != '':
                    text_bigrams.append(bg_r)
                    text_bigrams.append(bg_l)
    
        all_grams.append(text_bigrams)

    return all_grams   

In [545]:
n_grams_ent = got_ner_grams(all_clean_texts)

In [546]:
df['bi_grams'] = n_grams_ent

In [547]:
# три-граммы для сущностей

def got_ner_grams_2(sentence, entities=ent_in_texts):
    
    all_grams = []
    new_data = zip(sentence, entities)
    
    for s, e in tqdm(new_data):
        text_bigrams = []
        new_s = s.replace('  ', ' ')
        sent = new_s.split(' ')
        for w in e:

            # чтобы были би-граммы допускаем отступы
            entit = str(' ' + w + ' ')

            if entit in new_s:
                start = new_s.find(entit)
                end = start + len(w) + 1
                # контексты
                right = new_s[:start].split(' ')
                left = new_s[end+1:].split(' ')
                try:
                    bg_r = (right[-2], right[-1], w)
                    bg_l = (w, left[0], left[1])

                    middle = (right[-1], w, left[0])

                    # исключаем штуки, когда в не + сосед пустые
                    if (bg_l[1] != '' and bg_r[0] != '' 
                        and bg_l[2] != '' and bg_r[1] != ''):
                        text_bigrams.append(bg_r)
                        text_bigrams.append(bg_l)
                        text_bigrams.append(middle)
                except :
                    pass
        
        all_grams.append(text_bigrams)

    return all_grams   

In [548]:
trigrams_texts = got_ner_grams_2(all_clean_texts)

In [549]:
df['tri_grams'] = trigrams_texts

In [550]:
df.tail()

,title,helpfulness,score,summary,text,enteties,clean_text,clean_enteties,bi_grams,tri_grams
78925,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,9/9,4.0,"great sound quality, good battery life, averag...",I got this headset because I like the SonyEric...,"[connect, Battery, A, headset, battery, Ericss...",i got headset i like sonyericsson brand price ...,"[connect, battery, a, headset, battery, ericss...","[(headset, connect), (connect, automatically),...","[(phone, headset, connect), (connect, automati..."
78926,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,2/2,4.0,Excellent quality at an affordable price,I have owned this product for over two weeks n...,"[Sony Ericsson, A, W, headset, charger, phones...",i owned product two weeks sony ericsson w710i ...,"[sony ericsson, a, w, headset, charger, phones...","[(weeks, Sony Ericsson), (Sony Ericsson, W710i...","[(two, weeks, Sony Ericsson), (Sony Ericsson, ..."
78927,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,0/0,5.0,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,I bought this item for my husband. He really l...,[item],i bought item husband he really likes he sheri...,[item],"[(bought, item), (item, husband)]","[(I, bought, item), (item, husband, He), (boug..."
78928,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,0/0,5.0,"works great, competitively priced",I bought this item after I shopped in the loca...,"[item, headset, battery, Cingular, phone]",i bought item i shopped local cingular store i...,"[item, headset, battery, cingular, phone]","[(bought, item), (item, I), (nice, headset), (...","[(I, bought, item), (item, I, shopped), (bough..."
78929,Sony Ericsson Bluetooth HBH-PV700 Headset - DP...,0/0,5.0,Sony Ericsson Bluetooth HBH-PV700 Headset,"I have tried a number of bluetooth earpieces, ...","[connect, bluetooth, phone, earpiece, product]",i tried number bluetooth earpieces one tops pe...,"[connect, bluetooth, phone, earpiece, product]","[(number, bluetooth), (bluetooth, earpieces), ...","[(tried, number, bluetooth), (bluetooth, earpi..."


### (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.).

Мне в данной задаче кажется важным не приводить то, что в н-граммах к нижнему регистру, поэтому, все, что именованные сущности оставим такими какие есть, а все, что выделилось в н-граммах сделаем нижним регистром.

In [551]:
all_bi_grams = []

for g in n_grams_ent:
    for b in g:
        w1 = b[0]
        w2 = b[1]
        if w1 not in fin_ent:
            w1 = w1.lower()
        if w2 not in fin_ent:
            w2 = w2.lower()
        all_bi_grams.append((w1, w2))

In [555]:
all_bi_grams[-100:-75]

[('bling', 'camera'),
 ('camera', 'mp3'),
 ('mp3', 'bluetooth'),
 ('bluetooth', 'whatever'),
 ('the', 'battery'),
 ('battery', 'life'),
 ('former', 'Nokia'),
 ('Nokia', 'loyalist'),
 ('well', 'One'),
 ('One', 'important'),
 ('bought', 'Samsung t219'),
 ('Samsung t219', 'replacement'),
 ('replacement', 'phone'),
 ('phone', 'existing'),
 ('jack', 'Samsung'),
 ('Samsung', 'sells'),
 ('interest', 'A'),
 ('A', 'couple'),
 ('phone', 'camera'),
 ('camera', 'bluetooth'),
 ('proprietary', 'headset'),
 ('headset', 'jack'),
 ('camera', 'bluetooth'),
 ('bluetooth', 'i'),
 ('doubles', 'charger')]

Вроде получилось неплохо, у нас выделяются штуки типа названий и того, к чему принадлежит объект (телефон, камера, наушники и т.д.)

Вообще, здесь можно было бы лемматизировать тексты (и я хотела это сделать еще в начале), но решила оставить препроцессинг на уровне удаления пунктуации, так как у меня есть ощущение (особенно, что касается s на конце слов), что с названиями это могло все испортить (а нам нужно выделить именно устройства типа  "Samsung Galaxy Watch" и просто "watch", при том, что разные модели устройств могут идти в каких-то нестандартных формах, я побоялась делать лемматизацию, а разные модели могут отличаться только окнчанием)

In [446]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
# trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [443]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [558]:
finder2 = BigramCollocationFinder.from_documents(all_bi_grams)

In [450]:
finder2

In [561]:
# из семинара стащила частотный фильтр
finder2.apply_freq_filter(10)

Тут мне было не очень понятно, как лучше это делать: смотреть на уже выделенные н-граммы или искать н-граммы просто в текстах без учета того, что мы собрали. Мне кажется, что лучше всего было бы сделать это на тех н-граммах (би-граммах), которые уже были извлечены нами ранее. Поэтому я смотрела на них, мне кажется в условии данной задачи это правильнее

#### likelihood_ratio

In [577]:
# выведу первые 50, чтобы не мешало
finder2.score_ngrams(bigram_measures.likelihood_ratio)[:50]

[(('battery', 'life'), 20231.796016069064),
 (('phone', 'phone'), 18082.83905979449),
 (('Sony', 'Ericsson'), 14336.404121248772),
 (('t', 'Mobile'), 14084.157095341809),
 (('cell', 'phone'), 12427.711728176233),
 (('Treo', '650'), 8770.958857478607),
 (('5', 'stars'), 7286.2469645553265),
 (('bluetooth', 'headset'), 7095.908023242313),
 (('Battery', 'life'), 6991.347652103896),
 (('car', 'charger'), 6051.774128261226),
 (('Voyager', '510'), 5884.195270105772),
 (('phone', 'product'), 4950.817054240096),
 (('product', 'phone'), 4854.4014627654815),
 (('USB', 'cable'), 4194.430561800289),
 (('headset', 'phone'), 4155.04488196241),
 (('battery', 'phone'), 4082.098860989531),
 (('plantronics', 'Voyager'), 3975.189226819174),
 (('SD', 'card'), 3752.59411328487),
 (('phone', 'headset'), 3660.5723499185506),
 (('phone', 'phones'), 3562.0721807702853),
 (('Plantronics Voyager', '510'), 3236.984905197439),
 (('phones', 'phone'), 3223.8975787640234),
 (('4', '5'), 3063.8737323502737),
 (('this'

#### PMI

In [565]:
finder2.score_ngrams(bigram_measures.pmi)[:50]

[(('stowaway', 'Ultra Slim'), 14.14424010358553),
 (('aliph', 'Jawbone Noise'), 13.910015921891738),
 (('Jawbone Noise', 'shield'), 13.333024132279162),
 (('Leopard', 'print'), 12.9120666614565),
 (('RIZR', 'z3'), 12.702730392945497),
 (('Palm Tungsten', 'e2'), 12.69749374457935),
 (('Honda', 'accord'), 12.468077788481528),
 (('Honda', 'civic'), 12.342546906397668),
 (('Motorola Mobile Phone', 'tools'), 12.247050909169312),
 (('flexible', 'Boom Headset'), 12.132167271284958),
 (('Panasonic', 'kx'), 11.754077272945722),
 (('655', 'DSP'), 11.729202604306689),
 (('earwave', 'Boom'), 11.712461930963272),
 (('Cardo Systems', 'scala'), 11.520255714035192),
 (('ppc', '6700'), 11.477663837310725),
 (('Home', 'depot'), 11.412168116115595),
 (('Silicone', 'Skin'), 11.39966132530945),
 (('ultra', 'Slim'), 11.375329857415435),
 (('Wi', 'fi'), 11.21476762727577),
 (('Logitech Mobile', 'freedom'), 11.121520027085447),
 (('Explorer', '330'), 11.051846499278641),
 (('Explorer', '340'), 11.042568685690

#### Dice

In [566]:
finder2.score_ngrams(bigram_measures.dice)[:50]

[(('stowaway', 'Ultra Slim'), 0.5614035087719298),
 (('aliph', 'Jawbone Noise'), 0.5428571428571428),
 (('Wi', 'fi'), 0.5333333333333333),
 (('Jawbone Noise', 'shield'), 0.5116279069767442),
 (('Voyager', '510'), 0.4412470023980815),
 (('Sony', 'Ericsson'), 0.43093649617000246),
 (('t', 'Mobile'), 0.4210388247639035),
 (('Logitech Mobile', 'freedom'), 0.36428571428571427),
 (('SD', 'card'), 0.3619861185264282),
 (('Plantronics Voyager', '510'), 0.3554778554778555),
 (('plantronics', 'Voyager'), 0.3383816529640848),
 (('ppc', '6700'), 0.33035714285714285),
 (('li', 'Ion'), 0.3021978021978022),
 (('flexible', 'Boom Headset'), 0.2982456140350877),
 (('RIZR', 'z3'), 0.29411764705882354),
 (('Treo', '650'), 0.29276513783556035),
 (('Samsung Galaxy', 'note'), 0.29080118694362017),
 (('Palm Tungsten', 'e2'), 0.25316455696202533),
 (('Dual', 'band'), 0.25287356321839083),
 (('Belt', 'Clip'), 0.25092250922509224),
 (('lithium', 'Ion'), 0.25),
 (('Battery', 'life'), 0.2475526049285473),
 (('Blue

#### Хи-квадрат

In [567]:
finder2.score_ngrams(bigram_measures.chi_sq)[:50]

[(('aliph', 'Jawbone Noise'), 292458.83110053156),
 (('stowaway', 'Ultra Slim'), 289695.3224897194),
 (('Wi', 'fi'), 275609.86358230334),
 (('Jawbone Noise', 'shield'), 226998.170489869),
 (('t', 'Mobile'), 179787.1344402095),
 (('Voyager', '510'), 174700.5752667685),
 (('Sony', 'Ericsson'), 164662.65377383662),
 (('SD', 'card'), 134881.75349438502),
 (('Logitech Mobile', 'freedom'), 113562.12364016681),
 (('li', 'Ion'), 111856.85822159881),
 (('Plantronics Voyager', '510'), 109663.93321851181),
 (('plantronics', 'Voyager'), 108973.06228185866),
 (('ppc', '6700'), 105470.71923300637),
 (('Honda', 'accord'), 101969.52284523832),
 (('RIZR', 'z3'), 99980.57126034568),
 (('Blue', 'ant'), 86834.38103630455),
 (('Treo', '650'), 81047.18330119137),
 (('battery', 'life'), 78984.98614720651),
 (('lithium', 'Ion'), 78706.39876501836),
 (('Leopard', 'print'), 77064.05398814661),
 (('Samsung Galaxy', 'note'), 76717.57747258781),
 (('flexible', 'Boom Headset'), 76288.58113809646),
 (('V', 'cast'), 

#### Выводы




*   Likelihood ratio кажется самым хорошим варинатом, если говорить о ранжировании (если просматривать выдачу топ результатов): у нас выдается много информации о телефонах, моделях и т.д. в самом верху (в других тоже этого много, но кажется, что мусора больше: появляются коллокации с номерами моделей в би-граммах, но выглядит это как-то не очень)

*   Новый пункт



### (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.

Возьмем лучший варинат из прошлого пункта

In [580]:
# выведу первые 500
l = finder2.score_ngrams(bigram_measures.likelihood_ratio)[:500]

In [592]:
# выделим товары, которые точно есть в датасете

group_d = {}

# мы могли взять те нер, которые выделили мы, но для упрощения можно так
device = ['phone', 'camera', 'watch', 'headset',
          'battery', 'CD', 'adapter', 'bluetooth',
          'card', 'car']


# пройдемся по нашим результатам и выделим колокации по блокам
for d in device:
    list_gr = []
    for el in l:
        if d in el[0]:
            list_gr.append(el[0])
    group_d[d] = list_gr

Для 5 товаров

In [593]:
group_d['watch']

[('watch', 'movies'), ('watch', 'tv'), ('watch', 'videos')]

In [594]:
group_d['adapter']

[('AC', 'adapter'),
 ('cassette', 'adapter'),
 ('adapter', 'phone'),
 ('phone', 'adapter'),
 ('power', 'adapter')]

In [597]:
group_d['phone']

[('phone', 'phone'),
 ('cell', 'phone'),
 ('phone', 'product'),
 ('product', 'phone'),
 ('headset', 'phone'),
 ('battery', 'phone'),
 ('phone', 'headset'),
 ('phone', 'phones'),
 ('phones', 'phone'),
 ('5', 'phone'),
 ('bluetooth', 'phone'),
 ('phone', 'battery'),
 ('phone', 'bluetooth'),
 ('phone', 'Motorola'),
 ('charger', 'phone'),
 ('phone', 'i'),
 ('phone', 'item'),
 ('item', 'phone'),
 ('phone', '5'),
 ('Motorola', 'phone'),
 ('phone', 'cell'),
 ('A', 'phone'),
 ('USB', 'phone'),
 ('got', 'phone'),
 ('phone', 'camera'),
 ('phone', 'charger'),
 ('Treo', 'phone'),
 ('phone', 'Nokia'),
 ('life', 'phone'),
 ('earpiece', 'phone'),
 ('phone', 'Treo'),
 ('adapter', 'phone'),
 ('phone', 'earpiece'),
 ('Sony', 'phone'),
 ('phone', 'adapter'),
 ('One', 'phone'),
 ('phone', 'calls'),
 ('phone', 'A'),
 ('phone', 'USB'),
 ('Palm', 'phone'),
 ('this', 'phone'),
 ('bought', 'phone'),
 ('phone', 'it'),
 ('Cingular', 'phone'),
 ('Verizon', 'phone'),
 ('camera', 'phone'),
 ('phone', 'RAZR'),
 ('ph

In [596]:
group_d['camera']

[('the', 'camera'),
 ('digital', 'camera'),
 ('camera', 'takes'),
 ('phone', 'camera'),
 ('camera', 'video'),
 ('megapixel', 'camera'),
 ('camera', 'lens'),
 ('camera', 'battery'),
 ('camera', 'phone'),
 ('built', 'camera'),
 ('i', 'camera'),
 ('2mp', 'camera'),
 ('phones', 'camera'),
 ('camera', 'charger'),
 ('camera', 'mp3'),
 ('vga', 'camera'),
 ('camera', 'button'),
 ('mp', 'camera'),
 ('bluetooth', 'camera')]

In [598]:
group_d['headset']

[('bluetooth', 'headset'),
 ('headset', 'phone'),
 ('phone', 'headset'),
 ('battery', 'headset'),
 ('headset', 'headset'),
 ('headset', 'battery'),
 ('headset', 'product'),
 ('headset', 'i'),
 ('phones', 'headset'),
 ('headset', 'phones'),
 ('wireless', 'headset'),
 ('this', 'headset'),
 ('bt', 'headset'),
 ('headset', 'bluetooth'),
 ('5', 'headset'),
 ('headset', 'charger'),
 ('headset', 'works'),
 ('charger', 'headset'),
 ('wired', 'headset'),
 ('headset', '5'),
 ('headset', 'Motorola'),
 ('Motorola', 'headset'),
 ('using', 'headset'),
 ('Nokia', 'headset'),
 ('stereo', 'headset'),
 ('headset', 'comfortable'),
 ('A', 'headset'),
 ('bought', 'headset'),
 ('tooth', 'headset'),
 ('headset', 'A'),
 ('i', 'headset'),
 ('headset', 'Nokia'),
 ('headset', 'it'),
 ('headset', 'Samsung'),
 ('USB', 'headset'),
 ('headset', 'Verizon'),
 ('Samsung', 'headset'),
 ('headset', 'USB')]

Для всех всех неров

In [604]:
l_2 = finder2.score_ngrams(bigram_measures.likelihood_ratio)

In [608]:
# выделим товары, которые точно есть в датасете

group_d_extra = {}

# пройдемся по нашим результатам и выделим колокации по блокам
for d in fin_ent:
    list_gr = []
    for el in l_2:
        if d in el[0]:
            # print(d)
            list_gr.append(el[0])
            l_2.remove(el)
    if len(list_gr) >= 1:
        group_d_extra[d] = list_gr

In [617]:
# вывод для всех
for k, v in group_d_extra.items():
    cnt = 0
    if cnt <= 1:
        print(k, v)
    cnt += 1

connect [('connect', 'computer')]
Samsung [('Samsung', 'blackjack'), ('owned', 'Samsung')]
A [('A', 'c'), ('its', 'A')]
black [('shiny', 'black'), ('red', 'black')]
Sprint [('went', 'Sprint')]
camera [('vga', 'camera'), ('camera', '1'), ('camera', 'fine'), ('even', 'camera')]
item [('item', 'this'), ('item', 'advertised'), ('item', 'defective'), ('item', 'still'), ('item', 'like')]
headset [('headset', 'headset'), ('bt', 'headset'), ('headset', 'ear'), ('one', 'headset'), ('headset', 'based'), ('got', 'headset'), ('headset', 'performs'), ('headset', 'money'), ('headset', 'quite'), ('headset', 'seems'), ('included', 'headset'), ('headset', 'otherwise'), ('headset', 'finally'), ('headset', 'today'), ('ear', 'headset'), ('headset', 'however'), ('headset', 'they'), ('good', 'headset'), ('little', 'headset'), ('headset', 'though'), ('headset', 'wireless'), ('satisfied', 'headset'), ('headset', 'thought'), ('purchase', 'headset'), ('price', 'headset'), ('pair', 'headset'), ('need', 'headset'